In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [5]:
custom_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
}

In [42]:
response = requests.get("https://www.amazon.com/Dog-Man-Scarlet-Shedder-Underpants/dp/1338896431/ref=sr_1_4?dib=eyJ2IjoiMSJ9.k4N5imkSD5ERmBAlVVAQSCes5KdVI7Vrj7obtGmBcJ6W-C4uZxaSw--cHtw2ZazsdK5NakpVg_MmEsZI_R3y77ri8BLOmImojIovDBwqfnLAE37EeBt282OBGCJaFol1KmGIDhQsmHQP87LZL3PFrmlscZvHBclgjDkSYHCiA0xkT3zcINaDteGzbKp9sbVzWA4sDDNdf1Rniz_vwdTXZv_TwNdV2WCsf-tPcHZ4mOw.1G1qP9VTfq9oSmfKRiaem8QOAR8GGQakCqeh8IDFpoc&dib_tag=se&qid=1713040403&refinements=p_72%3A1250222011&s=books&sr=1-4", headers=custom_headers)
if response.status_code != 200:
    print(f"Error in getting webpage")
print(response.text)
soup = BeautifulSoup(response.text, "lxml")

title_element = soup.select_one("#productTitle")
title = title_element.text.strip() if title_element else None

binding_element = soup.select_one("#productBinding")
binding = binding_element.text.strip() if binding_element else None

image_element = soup.select_one("#landingImage")
image = image_element.attrs.get("src") if image_element else None

print(title, binding, image)

<!DOCTYPE html>
<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta http-equiv="content-type" content="text/html; charset=UTF-8">
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<title dir="ltr">Amazon.com</title>
<meta name="viewport" content="width=device-width">
<link rel="stylesheet" href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css">
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER",
    

In [10]:
visited_urls = set()

def get_product_info(url):
    response = requests.get(url, headers=custom_headers)
    if response.status_code != 200:
        print(f"Error in getting webpage: {url}")
        return None

    soup = BeautifulSoup(response.text, "lxml")

    title_element = soup.select_one("#productTitle")
    title = title_element.text.strip() if title_element else None

    binding_element = soup.select_one("#productBinding")
    binding = binding_element.text.strip() if binding_element else None

    image_element = soup.select_one("#landingImage")
    image = image_element.attrs.get("src") if image_element else None

    return {
        "title": title,
        "binding": binding,
        "image": image,
        "url": url
    }

In [14]:

def parse_listing(listing_url):
    global visited_urls
    response = requests.get(listing_url, headers=custom_headers)
    print(response.status_code)
    soup_search = BeautifulSoup(response.text, "lxml")
    link_elements = soup_search.select("[data-asin] h2 a")
    page_data = []
    for link in link_elements:
        full_url = urljoin(listing_url, link.attrs.get("href"))
        if full_url not in visited_urls:
            visited_urls.add(full_url)

            print(f"Scraping product from {full_url[:100]}", flush=True)
            product_info = get_product_info(full_url)
            if product_info:
                page_data.append(product_info)

    next_page_el = soup_search.select_one('a.s-pagination-next')
    if next_page_el:
        next_page_url = next_page_el.attrs.get('href')
        next_page_url = urljoin(listing_url, next_page_url)
        print(f'Scraping next page: {next_page_url}', flush=True)
        page_data += parse_listing(next_page_url)

    return page_data

In [18]:
data = []
search_url = "https://www.amazon.com/s?i=stripbooks&bbn=283155&rh=n%3A283155%2Cp_72%3A1250222011"
data = parse_listing(search_url)
df = pd.DataFrame(data)
df.to_csv("books.csv")

200
Scraping product from https://www.amazon.com/Women-Novel-Kristin-Hannah/dp/1250178630/ref=sr_1_1?dib=eyJ2IjoiMSJ9.k4N5imkS
Scraping product from https://www.amazon.com/Merriam-Websters-Spanish-English-Dictionary-Mass-Market-Multilingual/dp/08777
Scraping product from https://www.amazon.com/Anxious-Generation-Rewiring-Childhood-Epidemic/dp/0593655036/ref=sr_1_3?dib=e
Scraping product from https://www.amazon.com/Dog-Man-Scarlet-Shedder-Underpants/dp/1338896431/ref=sr_1_4?dib=eyJ2IjoiMSJ9.
Scraping product from https://www.amazon.com/Court-Thorns-Roses-Paperback-books/dp/1639730192/ref=sr_1_5?dib=eyJ2IjoiMSJ9.
Scraping product from https://www.amazon.com/Atomic-Habits-Proven-Build-Break/dp/0735211299/ref=sr_1_6?dib=eyJ2IjoiMSJ9.k4
Scraping product from https://www.amazon.com/Fourth-Wing-Empyrean-Rebecca-Yarros/dp/1649374046/ref=sr_1_7?dib=eyJ2IjoiMSJ9
Scraping product from https://www.amazon.com/Oh-Places-Youll-Dr-Seuss/dp/0679805273/ref=sr_1_8?dib=eyJ2IjoiMSJ9.k4N5imkSD5
Scraping pro